# Regression of Vote Characteristics on CAR

In [1]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      StataNow 19.5
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2025 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-782-8272        https://www.stata.com
                                   979-696-4600        service@stata.com

Stata license: Unlimited-user 2-core network, expiring  9 Sep 2026
Serial number: 501909305069
  Licensed to: 罗奕辰
               UCL

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


In [2]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [3]:
%%stata
foreach stage in created end {
    import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

    * Parse date
    gen day = date(date, "YMD")
    format day %td

    * Create year variable
    gen year = year(day)

    * Create quarter variable
    gen quarter = qofd(day)
    format quarter %tq

    * Ensure index is numeric (safeguard if it came in as string)
    capture confirm numeric variable index
    if _rc {
        destring index, replace ignore("[]")
    }

    * Encode categorical variables
    encode gecko_id, gen(token)
    encode space,    gen(dao)

    egen token_year = group(token year)
    egen token_quarter = group(token quarter)

    * Define windows and specs
    local window 5
    local vars    non_whale_victory_vn non_whale_victory_vp non_whale_victory_vp_vn

    * Label variables
    label var non_whale_victory_vn "\${\it Small Win}^{vn}_{i,t}\$"
    label var non_whale_victory_vp "\${\it Small Win}^{vp}_{i,t}\$"
    label var non_whale_victory_vp_vn "\${\it Small Win}^{vp,vn}_{i,t}\$"

    eststo clear

    keep if index == `window'

    foreach v of local vars {

        * Run regression
        reghdfe car `v', absorb(year dao) vce(cluster token)
        eststo car_`v'

        * Tag FE rows (Y/N) for this spec
        estadd local fe_token "Y"
        estadd local fe_time  "Y"
    }

    * Export a single big LaTeX table
    esttab                                                         ///
        car_non_whale_victory_vn car_non_whale_victory_vp car_non_whale_victory_vp_vn ///
        using "$TABLES/reg_car_win_`stage'.tex", replace         ///
        se star(* 0.10 ** 0.05 *** 0.01) label nogaps nocon        ///
        nonotes booktabs nomtitles                                 ///
        b(%9.3f) se(%9.2f)                                         ///
        mgroups("CAR[-5,5]",                                       ///
                span                                               ///
                pattern(1 0 0)                                     ///
                prefix(\multicolumn{@span}{c}{)                    ///
                suffix(})                                          ///
                erepeat(\cmidrule(lr){@span}) )                    ///
        substitute("\_" "_")                                       ///
        stats(fe_token fe_time N r2_a,                             ///      
              fmt(0 0 %9.0fc %9.3f) labels("Token FE" "Year FE" "Observations" "Adjusted R²"))
}


. foreach stage in created end {
  2.     import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv"
> , clear
  3. 
.     * Parse date
.     gen day = date(date, "YMD")
  4.     format day %td
  5. 
.     * Create year variable
.     gen year = year(day)
  6. 
.     * Create quarter variable
.     gen quarter = qofd(day)
  7.     format quarter %tq
  8. 
.     * Ensure index is numeric (safeguard if it came in as string)
.     capture confirm numeric variable index
  9.     if _rc {
 10.         destring index, replace ignore("[]")
 11.     }
 12. 
.     * Encode categorical variables
.     encode gecko_id, gen(token)
 13.     encode space,    gen(dao)
 14. 
.     egen token_year = group(token year)
 15.     egen token_quarter = group(token quarter)
 16. 
.     * Define windows and specs
.     local window 5
 17.     local vars    non_whale_victory_vn non_whale_victory_vp non_whale_vict
> ory_vp_vn
 18. 
.     * Label variables
.     label var non_whale_victory_vn "\${\it